# Pipeline to run all files
* ~~All scripts currently output Excel files because we were working independently~~
* ~~<b>We'll be changing this so that each script passes dataframes until the end of the pipeline.</b>~~
* Our plan is to not store anything (refresh daily) but need to consider how to deal with taking in user feedback

In [1]:
#Gather Articles
import NewsAPI as news
import os

#delete any old sorted article files
#if os.path.exists("articles.csv"):
#    os.remove("articles.csv")

#inputs in order: set manual date (yes=1/no=0, no = last 24 hours of news), manual start date as "YYYY-MM-DD"(leave empty if 0), manual end date (leave empty if 0, companies to pull (6= all 19 companies)
articleDB = news.main(1,"2019-02-28","2019-03-03",6) #output is called 'NewsAPIOutput.xlsx' in Python Scripts > Data folder

Gathering articles on (Gap Inc) OR (Foot Locker) OR (L Brands) OR Macerich OR Kimco OR TJX OR CVS OR (Home Depot) OR (Best Buy) OR (Lowe's) OR Walmart OR (Target's) OR TGT OR Amazon OR Kroger OR Walgreens OR Kohl's OR (Dollar General) OR (Bed Bath and Beyond) OR Safeway from: 2019-02-28 to 2019-03-03
534


In [2]:
#Article Cleaning (must pip install tqdm first (only once) to run)
import dataClean as dc
articleDB = dc.DataClean(articleDB)

In [3]:
#Feature Selection and Binary Article Encoding
import FeatureEncoding as fe
binaryMatrix = fe.encoding(0, df=articleDB)

Binary Encoding


100%|██████████| 319/319 [00:00<00:00, 3989.14it/s]


In [4]:
#Logistic Regression Classifier + Article Ranking, complete final file is called 'results_encoding.xlsx'
import logReg as lr
articleDB = lr.runLogReg(binaryMatrix, articleDB)

C:\Users\Legendary\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


                                               title  \
0  Trump-Kim summit ends abruptly with no deal: '...   
1  'We must be saints,' young Catholic says about...   
2      What to look for in a usage and grammar guide   
3  New data sources make it harder for executives...   
4  Bill Gross says this secret condition made him...   

                                         description  
0  Plus, 'impressed' Trump slams 'fake' Michael C...  
1  Being Catholic is one of the most unpopular th...  
2  “Dreyer’s English”, a bestselling book, is rar...  
3                       How to cross-check Elon Musk  
4  Even after one of the most storied careers in ...  


In [5]:
# This code extracts and ranks "tags" + counts frequency of tag mentions in articles 
"""
Inputs: 
    1) articleDB - uses column 'content'
    2) (optional) - tag type (could be 'ngrams'{unlimited}, 'bigrams'{terms with up to 2 words}, or 'unigrams'{single terms})
        - default is 'bigrams'
        - future work: add in noun phrases, named entities 
Outputs:
    1) articleDB = articleDB with appended columns "tags" and "tags_top_5"
    2) trendingTermsDB = keyterms by # article mentions
"""
import ContextExtraction as ce
articleDB, trendingTermsDB = ce.retrieveContext(articleDB)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Legendary\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Legendary\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


100%|██████████| 319/319 [00:19<00:00, 11.23it/s]


In [6]:
articleDB.head()

,nonRel,Rel,url,prediction,title,description,source,date,content,origContent,contentWithStops,tags,tags_top_5
0,0.133170,0.866830,https://www.foxnews.com/us/trump-kim-summit-en...,1,Trump-Kim summit ends abruptly with no deal: '...,"Plus, 'impressed' Trump slams 'fake' Michael C...",fox-news,2019-02-28T10:06:42Z,deal president trump abruptly walked away neg...,NO DEAL: President Trump abruptly walked away ...,deal president trump abruptly walked away neg...,"president trump, away negotiations, back washi...","personal lawyer,first lady,convicted felon,awa..."
1,0.272690,0.727310,https://www.cbc.ca/news/canada/manitoba/young-...,1,"'We must be saints,' young Catholic says about...",Being Catholic is one of the most unpopular th...,cbc-news,2019-03-03T12:00:00Z,abuse summit took place rome address global p...,"In February 2019, an abuse summit took place i...","2019, abuse summit took place rome address gl...","abuse summit, catholic church, roland lanoie, ...","great millstone,treated seminarian,abuse summi..."
2,0.375336,0.624664,https://www.economist.com/books-and-arts/2019/...,1,What to look for in a usage and grammar guide,"“Dreyer’s English”, a bestselling book, is rar...",the-economist,2019-02-28T15:55:48Z,people buy books english usage obvious answer...,WHY DO PEOPLE buy books on English usage? The ...,people buy books english usage? obvious answe...,"recent edition, small pagesnor, ms truss, dail...","informed decades,serious stuff,new economist,d..."
3,0.386786,0.613214,https://www.economist.com/finance-and-economic...,1,New data sources make it harder for executives...,How to cross-check Elon Musk,the-economist,2019-02-28T15:58:13Z,go straight useful rule anyone seeking accurat...,GO STRAIGHT to the source is a useful rule for...,go straight useful rule anyone seeking accurat...,"caused firms, false sec, teslas chairman, chie...","teslas chairman,research clients,available inv..."
4,0.393527,0.606473,https://business.financialpost.com/investing/t...,1,Bill Gross says this secret condition made him...,Even after one of the most storied careers in ...,financial-post,2019-03-01T19:46:01Z,even one storied careers financial markets bil...,Even after one of the most storied careers in ...,"even one storied careers financial markets, bi...","successful investor, moribund economy, us gove...","decade marriage,feelings pimco,official retire..."


In [7]:
import EncodingforRecommender as rec
finalTable = rec.recommender(articleDB)

17954


['deal', 'president', 'trump', 'abruptly', 'walked', 'away', 'negotiations', 'with', 'north', 'korean', 'leader', 'kim', 'jong', 'un', 'vietnam', 'headed', 'back', 'washington', 'saying', 'unwilling', 'meet', 'demand', 'lifting', 'sanctions', 'rogue', 'regime', 'without', 'first', 'securing', 'meaningful', 'commitment', 'denuclearization', 'speaking', 'hanoi', 'asked', 'regarding', 'intentions', 'denuclearize', 'unprepared', 'sometimes', 'walk', 'summit', 'fell', 'demanded', 'full', 'removal', 'led', 'international', 'exchange', 'shuttering', 'yongbyon', 'nuclear', 'facility', 'secretary', 'state', 'mike', 'pompeo', 'told', 'reporters', 'willing', 'make', 'committing', 'giving', 'up', 'secretive', 'facilities', 'outside', 'well', 'missile', 'warheads', 'program', 'promised', 'would', 'resume', 'testing', 'taking', 'word', 'convicted', 'liar', 'cohen', 'racist', 'con', 'man', 'cheat', 'michael', 'former', 'fixer', 'begin', 'three', 'year', 'prison', 'term', 'denounced', 'explosive', 'co

tifidf Encoding
bin Encoding


100%|██████████| 319/319 [00:01<00:00, 245.33it/s]


tf Encoding


100%|██████████| 319/319 [00:01<00:00, 277.58it/s]


         deal  president     trump  abruptly    walked      away  \
0    0.083333   0.116505  0.351852     0.125  0.142857  0.014925   
1    0.011905   0.000000  0.000000     0.000  0.000000  0.000000   
2    0.000000   0.000000  0.000000     0.000  0.000000  0.000000   
3    0.000000   0.000000  0.000000     0.000  0.000000  0.000000   
4    0.000000   0.009709  0.037037     0.000  0.000000  0.014925   
5    0.011905   0.000000  0.000000     0.000  0.000000  0.000000   
6    0.000000   0.000000  0.000000     0.000  0.000000  0.000000   
7    0.000000   0.000000  0.000000     0.000  0.000000  0.000000   
8    0.035714   0.213592  1.629630     0.125  0.000000  0.014925   
9    0.000000   0.000000  0.000000     0.000  0.000000  0.000000   
10   0.000000   0.019417  0.240741     0.000  0.000000  0.014925   
11   0.000000   0.019417  0.185185     0.000  0.000000  0.014925   
12   0.000000   0.009709  0.000000     0.000  0.000000  0.000000   
13   0.011905   0.029126  0.055556     0.000  0.

[319 rows x 17954 columns]


In [8]:
finalTable.head()

,nonRel,Rel,url,prediction,title,description,source,date,content,origContent,contentWithStops,tags,tags_top_5,related_articles
0,0.133170,0.866830,https://www.foxnews.com/us/trump-kim-summit-en...,1,Trump-Kim summit ends abruptly with no deal: '...,"Plus, 'impressed' Trump slams 'fake' Michael C...",fox-news,2019-02-28T10:06:42Z,deal president trump abruptly walked away neg...,NO DEAL: President Trump abruptly walked away ...,deal president trump abruptly walked away neg...,"president trump, away negotiations, back washi...","personal lawyer,first lady,convicted felon,awa...","45, 209, 8"
1,0.272690,0.727310,https://www.cbc.ca/news/canada/manitoba/young-...,1,"'We must be saints,' young Catholic says about...",Being Catholic is one of the most unpopular th...,cbc-news,2019-03-03T12:00:00Z,abuse summit took place rome address global p...,"In February 2019, an abuse summit took place i...","2019, abuse summit took place rome address gl...","abuse summit, catholic church, roland lanoie, ...","great millstone,treated seminarian,abuse summi...","161, 74, 34"
2,0.375336,0.624664,https://www.economist.com/books-and-arts/2019/...,1,What to look for in a usage and grammar guide,"“Dreyer’s English”, a bestselling book, is rar...",the-economist,2019-02-28T15:55:48Z,people buy books english usage obvious answer...,WHY DO PEOPLE buy books on English usage? The ...,people buy books english usage? obvious answe...,"recent edition, small pagesnor, ms truss, dail...","informed decades,serious stuff,new economist,d...","224, 27, 109"
3,0.386786,0.613214,https://www.economist.com/finance-and-economic...,1,New data sources make it harder for executives...,How to cross-check Elon Musk,the-economist,2019-02-28T15:58:13Z,go straight useful rule anyone seeking accurat...,GO STRAIGHT to the source is a useful rule for...,go straight useful rule anyone seeking accurat...,"caused firms, false sec, teslas chairman, chie...","teslas chairman,research clients,available inv...","72, 307, 195"
4,0.393527,0.606473,https://business.financialpost.com/investing/t...,1,Bill Gross says this secret condition made him...,Even after one of the most storied careers in ...,financial-post,2019-03-01T19:46:01Z,even one storied careers financial markets bil...,Even after one of the most storied careers in ...,"even one storied careers financial markets, bi...","successful investor, moribund economy, us gove...","decade marriage,feelings pimco,official retire...","59, 88, 37"


In [9]:
import frontPage as fp
fp.FrontPage(articleDB, trendingTermsDB)

In [10]:
finalTable.to_excel("march3rdRun.xlsx")